In [ ]:
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from libsync.spotify.spotify_api_utils import (
    get_spotify_song_details,
    get_user_playlists_details,
)
import nest_asyncio
from libsync.utils.string_utils import (
    get_spotify_uri_from_id,
    pretty_print_spotify_track,
)

# hardcode playlist ID
playlist_id = "0w888XrmxysfqkgJrs5X0P"
playlist_id = "5AO34psq7NfH7zhFHOjz4P"
playlist_all_liked_songs_id = "0bRnlUkMje1OvCwVspaQYV"
playlist_id = playlist_all_liked_songs_id

# get secrets
load_dotenv()

# spotify auth
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=auth_manager)

# allow nested async (asyncio inside jupyter)
nest_asyncio.apply()

# get playlist details
playlist_details = get_user_playlists_details([playlist_id])

# get song details
uris = [get_spotify_uri_from_id(id) for id in playlist_details[playlist_id]]
songs = get_spotify_song_details(uris)

In [ ]:
# DO STUFF WITH THE DATA (no network calls here)

# Add a function to format track duration from milliseconds to mm:ss format
def format_duration(ms):
    seconds = ms // 1000
    minutes = seconds // 60
    seconds = seconds % 60
    return f"{minutes}:{seconds:02d}"

# Add a function to extract and print relevant track information
def print_track_info(track_data):
    # Parse the track data from string if needed
    if isinstance(track_data, str):
        try:
            track_data = eval(track_data)  # Using eval since the data appears to be a Python dict representation
        except:
            print("Error parsing track data")
            return
    
    # Extract relevant information
    track_name = track_data.get('name', 'Unknown Track')
    artist_name = track_data.get('artists', [{}])[0].get('name', 'Unknown Artist')
    duration_ms = track_data.get('duration_ms', 0)
    duration_formatted = format_duration(duration_ms)
    
    # Print formatted information
    print(f"Track: {track_name}")
    print(f"Artist: {artist_name}")
    print(f"Duration: {duration_formatted}")
    print("-" * 50)


tracks = list(songs.values())
# sort tracks by duration
tracks.sort(key=lambda x: x['duration_ms'], reverse=True)

# filter tracks to 8-12 minutes
tracks = [t for t in tracks if 8*60*1000 <= t['duration_ms'] <= 12*60*1000]
print(f"\nNumber of tracks between 8-12 minutes: {len(tracks)}\n")


# pretty print
for track in tracks:
    print(pretty_print_spotify_track(track) + " " + format_duration(track['duration_ms']))

In [ ]:
# disambiguate tracks, then add them all to a playlist

# disambiguate tracks, then add them all to a playlist

new_tracks = []
SIMILARITY_THRESHOLD = 0.9

def calculate_track_similarity(track1, track2):
    """Calculate similarity between two Spotify tracks"""
    # Extract track names and artists for comparison
    track1_name = track1.get('name', '').lower()
    track2_name = track2.get('name', '').lower()
    
    track1_artists = [artist['name'].lower() for artist in track1.get('artists', [])]
    track2_artists = [artist['name'].lower() for artist in track2.get('artists', [])]
    
    # Calculate name similarity using SequenceMatcher
    from difflib import SequenceMatcher
    name_similarity = SequenceMatcher(None, track1_name, track2_name).ratio()
    
    # Calculate artist similarity - find best match between artists
    artist_similarities = []
    for artist1 in track1_artists:
        for artist2 in track2_artists:
            artist_similarities.append(SequenceMatcher(None, artist1, artist2).ratio())
    
    artist_similarity = max(artist_similarities) if artist_similarities else 0
    
    # Overall similarity is a combination of name and artist similarity
    return name_similarity * artist_similarity

for track in tracks:
    # Check if this track is too similar to any already added track
    is_duplicate = False
    for existing_track in new_tracks:
        similarity = calculate_track_similarity(track, existing_track)
        if similarity > SIMILARITY_THRESHOLD:
            print(f"Skipping similar track: {pretty_print_spotify_track(track)} (similarity: {similarity:.2f})")
            print(f"  Similar to: {pretty_print_spotify_track(existing_track)}")
            is_duplicate = True
            break
    
    if not is_duplicate:
        new_tracks.append(track)
        # print(pretty_print_spotify_track(track) + " " + format_duration(track['duration_ms']))
        # print_track_info(track)
        # print("-" * 50)

print(f"\nTotal tracks after disambiguation: {len(new_tracks)} (removed {len(tracks) - len(new_tracks)} duplicates)")

for track in new_tracks:
  print(track['id'] + " " + pretty_print_spotify_track(track) + " " + format_duration(track['duration_ms']))

In [ ]:
# add all new tracks to a new spotify playlist

# add all new tracks to a new spotify playlist
from libsync.spotify.spotify_auth import SpotifyAuthManager
from libsync.spotify import spotify_api_utils

# Create a new playlist
playlist_name = "Long Songs (8-12 min)"
playlist_description = "Songs between 8-12 minutes long, automatically generated by libsync"

# Get authenticated Spotify client that can modify playlists
spotify = SpotifyAuthManager.get_spotify_client()
user_id = SpotifyAuthManager.get_user_id()

# Create the playlist
print(f"Creating new playlist: {playlist_name}")
playlist_create_result = spotify.user_playlist_create(
    user=user_id,
    name=playlist_name,
    public=False,  # Make it public
    description=playlist_description
)

playlist_id = playlist_create_result["id"]
print(f"Created playlist with ID: {playlist_id}")

# Extract track URIs from the filtered tracks
track_uris = [track['uri'] for track in new_tracks]
print(f"Adding {len(track_uris)} tracks to playlist...")

# Use the existing function from libsync to add tracks to the playlist
# This handles pagination and rate limiting automatically
spotify_playlist_write_jobs = [[playlist_id, track_uris]]
responses = spotify_api_utils.overwrite_playlists(spotify_playlist_write_jobs)

print(f"Successfully added tracks to playlist: {playlist_name}")
print(f"Playlist URL: https://open.spotify.com/playlist/{playlist_id}")